## Bangkit Final Project: Aksara Jawa Classification with Machine Learning

## Dependencies


In [0]:
# Install dependencies

!pip install -q kaggle
!pip install keras==2.3.0

     |████████████████████████████████| 378kB 8.6MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [2]:
import keras,os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Colab library to upload files to notebook
from google.colab import files

Using TensorFlow backend.


In [0]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

## Data Preparation


In [3]:
%%time
# Download dataset from upstream
!kaggle datasets download -d phiard/aksara-jawa

 94% 175M/186M [00:01<00:00, 89.9MB/s]
100% 186M/186M [00:01<00:00, 102MB/s] 
CPU times: user 24.6 ms, sys: 12.8 ms, total: 37.4 ms
Wall time: 5.75 s


In [4]:
%%time
# Unzip dataset into a folder
!unzip aksara-jawa.zip -d dataset

Archive:  aksara-jawa.zip
  inflating: dataset/dataset/newtesting/ba/ba178.pred.png  
  inflating: dataset/dataset/newtesting/ba/ba198.pred.png  
  inflating: dataset/dataset/newtesting/ba/ba218.pred.png  
  inflating: dataset/dataset/newtesting/ca/ca163.pred.png  
  inflating: dataset/dataset/newtesting/ca/ca183.pred.png  
  inflating: dataset/dataset/newtesting/ca/ca203.pred.png  
  inflating: dataset/dataset/newtesting/da/da166.pred.png  
  inflating: dataset/dataset/newtesting/da/da186.pred.png  
  inflating: dataset/dataset/newtesting/da/da206.pred.png  
  inflating: dataset/dataset/newtesting/dha/dha172.pred.png  
  inflating: dataset/dataset/newtesting/dha/dha192.pred.png  
  inflating: dataset/dataset/newtesting/dha/dha212.pred.png  
  inflating: dataset/dataset/newtesting/ga/ga177.pred.png  
  inflating: dataset/dataset/newtesting/ga/ga197.pred.png  
  inflating: dataset/dataset/newtesting/ga/ga217.pred.png  
  inflating: dataset/dataset/newtesting/ha/ha161.pred.png  
  inflat

In [12]:
# Do the image augmentation and rescale image
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    directory="/content/dataset/newdatasetaug/newtraining",
    target_size=(224,224),
    batch_size=20)

validation_data = validation_datagen.flow_from_directory(
    directory="/content/dataset/newdatasetaug/newtesting",
    target_size=(224,224),
    batch_size=20)

Found 1441 images belonging to 20 classes.
Found 839 images belonging to 20 classes.


## Build the Model



In [13]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=20, activation="softmax"))

from keras.optimizers import Adam
opt = Adam(lr=0.000001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

model.summary()

from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=50,generator=train_data, validation_data= validation_data, validation_steps=5,epochs=50,callbacks=[checkpoint,early])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 56, 56, 256)      